In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
file_path = 'ANN-2.xlsx'

In [3]:
data = pd.read_excel(file_path)

In [4]:
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Sliding velocity,NaN,Normal load,Filler content,Sliding distance,Specific wear rate
4,NaN,NaN,NaN,NaN,NaN,A (m/s),B (N),C (%),D (m),Ws (mm3/N-km)


In [5]:
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'])

In [6]:
data.columns = data.iloc[3]
data = data.drop(index=[0, 1, 2, 3])


In [7]:
data = data.dropna(axis=1, how='all')

In [8]:
data.reset_index(drop=True, inplace=True)
data = data.apply(pd.to_numeric, errors='coerce')

In [9]:
data.head()

3,NaN,Normal load,Filler content,Sliding distance,Specific wear rate
0,NaN,NaN,NaN,NaN,NaN
1,0.5,10.0,5.0,500.0,0.132890
2,0.5,10.0,10.0,1000.0,0.113821
3,0.5,10.0,15.0,1500.0,0.074015
4,0.5,20.0,5.0,1000.0,0.161960


In [10]:
data.isnull().sum()

3
NaN                   1
Normal load           1
Filler content        1
Sliding distance      1
Specific wear rate    1
dtype: int64

In [11]:
data.dropna(inplace=True)

In [12]:
data.shape

(27, 5)

In [13]:
data.rename(columns={data.columns[0]: 'A (m/s)'}, inplace=True)


In [14]:
data.head()

3,A (m/s),Normal load,Filler content,Sliding distance,Specific wear rate
1,0.5,10.0,5.0,500.0,0.132890
2,0.5,10.0,10.0,1000.0,0.113821
3,0.5,10.0,15.0,1500.0,0.074015
4,0.5,20.0,5.0,1000.0,0.161960
5,0.5,20.0,10.0,1500.0,0.132791


In [15]:
x=data.drop("Specific wear rate",axis=1)
y=data["Specific wear rate"]

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [17]:
def pushpa(model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error (MAE): {mae}")
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error (MSE): {mse}")
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    list_1 = []
    for a in x_train.columns:
        while True:
            try:
                value = input(f"enter the value of {a} : ")
                if value.strip() == "":
                    raise ValueError("Input cannot be empty")
                list_1.append(float(value))
                break
            except ValueError as e:
                print(f"Invalid input: {e}. Please enter a valid number for {a}.")
    list = np.array(list_1).reshape(1, -1)
    result =model.predict(list)
    print(f"the predicted value is: {result[0]}")
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error (MAE): {mae}")
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error (MSE): {mse}")
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    list_1 = []
    for a in x_train.columns:
        while True:
            try:
                value = input(f"enter the value of {a} : ")
                if value.strip() == "":
                    raise ValueError("Input cannot be empty")
                list_1.append(float(value))
                break
            except ValueError as e:
                print(f"Invalid input: {e}. Please enter a valid number for {a}.")

    list = np.array(list_1).reshape(1, -1)
    result = model.predict(list)
    print(f"the predicted value is Ws (mm3/N-km): {result[0]}")

In [18]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
x_t=data.drop("Specific wear rate",axis=1)
Y=data["Specific wear rate"]

In [20]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(x_t)

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

In [22]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

C:\Users\GUNATEJA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [24]:
history = model.fit(X_train, Y_train, epochs=100, batch_size=10, validation_split=0.2, verbose=1)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 345ms/step - loss: 0.1746 - mae: 0.3512 - val_loss: 0.0933 - val_mae: 0.2555
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.1106 - mae: 0.2793 - val_loss: 0.0573 - val_mae: 0.1848
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 0.0669 - mae: 0.2204 - val_loss: 0.0332 - val_mae: 0.1530
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.0481 - mae: 0.1820 - val_loss: 0.0188 - val_mae: 0.1283
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 0.0257 - mae: 0.1294 - val_loss: 0.0117 - val_mae: 0.1050
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.0228 - mae: 0.1183 - val_loss: 0.0086 - val_mae: 0.0827
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.0187 - mae: 0.1101 - val_loss: 0.0075 - val_mae: 0.0687
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0171 - mae: 0.1089 - val_loss: 0.0068 - val_mae: 0.0638
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.0215 - m

In [25]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0088 - mae: 0.0822
Mean Absolute Error on test data: 0.08215215802192688


In [ ]:
import numpy as np
list_1 = []

for a in x_train.columns:
    while True:
        try:
            value = input(f"Enter the value for {a}: ")
            if value.strip() == "":
                raise ValueError("Input cannot be empty.")
            list_1.append(float(value))
            break
        except ValueError as e:
            print(f"Invalid input: {e}. Please enter a valid number for {a}.")
input_array = np.array(list_1).reshape(1, -1)
input_scaled = scaler.transform(input_array)
result = model.predict(input_scaled)
print(f"The predicted value is: {result[0][0]} Ws (mm³/N-km)")


In [ ]:
from sklearn.ensemble import RandomForestRegressor
pushpa(RandomForestRegressor())

Mean Absolute Error (MAE): 0.022883709999999974
Mean Squared Error (MSE): 0.0009826663664177667
Root Mean Squared Error (RMSE): 0.03134750973231792


C:\Users\GUNATEJA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


the predicted value is: 0.11876243999999993
Mean Absolute Error (MAE): 0.02254030833333331
Mean Squared Error (MSE): 0.0009560416988508492
Root Mean Squared Error (RMSE): 0.030919923978736576
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: Input cannot be empty. Please enter a valid number for Sliding distance.
Invalid input: I